In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import re
import csv
import glob, os

In [ ]:
for i,chunk in enumerate(pd.read_csv('trips.csv', chunksize=100000)):
    chunk.to_csv('chunks/trips_chunk{}.csv'.format(i))
    print("numOf:" + str(i))

In [ ]:
for i in range(296):
    df = pd.read_csv('chunks/trips_chunk{}.csv'.format(i))
    g=df.groupby('LinkId')
    for gp in g:
        directory = 'chunks/' + gp[0]
        if not os.path.exists(directory):
            os.makedirs(directory)
    
        filename = directory + '/' + 'chunk{}'.format(i) + '.csv'
        print(filename)
        gp[1].to_csv(filename)

In [ ]:
result = ''
reg_compile = re.compile('chunks/LAy_65-67/chunk[0-9]+')
for dirpath, dirnames, filenames in os.walk('chunks/'):
    print(dirpath)
    print(dirnames)
    #print(filenames)
    #result = result + [dirname for dirname in dirnames if  reg_compile.match(dirname)]

In [ ]:
#os.chdir("C:/Folder")
os.makedirs("clean")
reg_compile = re.compile(r'chunks\\(.*)\\')
found_dirs = glob("chunks/*/")
for tmp_counter, tmp_dir in enumerate(found_dirs):
    results = pd.DataFrame([])

    tmp_dir = re.findall(reg_compile, tmp_dir, flags=0)[0]
    print (tmp_dir)

    interesting_files = glob("chunks/"+tmp_dir+"/*.csv") 
    for counter, file in enumerate(interesting_files):
        print (file)
        namedf = pd.read_csv(file, skiprows=0, usecols=[2,3,4,5,6,7,8,9])
        results = results.append(namedf)
 
    results.to_csv('clean/'+tmp_dir+'.csv', index=False)

In [2]:
df = pd.read_csv('../data/clean/LPa_15-16.csv', skiprows=0)

In [3]:
df.head(5)

,LinkId,SrcDetectorId,DestDetectorId,PathDistance_m,TripStart_dt,TripEnd_dt,Speed_kmh,TravelTime_s
0,LPa_15-16,15,16,1235,2016-01-25 15:42:31,2016-01-25 15:46:00,21.272727,209
1,LPa_15-16,15,16,1235,2016-01-25 15:45:31,2016-01-25 15:48:25,25.551724,174
2,LPa_15-16,15,16,1235,2016-01-25 15:44:23,2016-01-25 15:48:27,18.221311,244
3,LPa_15-16,15,16,1235,2016-01-25 16:00:39,2016-01-25 16:04:29,19.330435,230
4,LPa_15-16,15,16,1235,2016-01-25 16:16:54,2016-01-25 16:20:49,18.919149,235


In [4]:
df['TripStart_dt'] =  pd.to_datetime(df['TripStart_dt'])
df['TripEnd_dt'] =  pd.to_datetime(df['TripEnd_dt'])

#df2 = df.head(1000000)
df2 = df

df3 = df2[['TripStart_dt', 'TripEnd_dt', 'Speed_kmh', 'TravelTime_s']]
the_time=df3['TripStart_dt'].dt.time
df3 = df3.assign(start_time=the_time.values)
the_day_num=df3['TripStart_dt'].dt.weekday
df3 = df3.assign(day_num=the_day_num.values)
df3 = df3.sort_values(by=['start_time'])

df4 = df3.loc[df3['day_num'] == 4] #0->Monday,6->sunday

df3['mean'] = df3['TravelTime_s'].rolling(1000).mean()
df4['mean'] = df4['TravelTime_s'].rolling(250).mean()


C:\Users\chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
SMALL_SIZE = 4
MEDIUM_SIZE = 4
BIGGER_SIZE = 6
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

ax1 = df4.plot(x='start_time',y='TravelTime_s',color='red',marker=',',lw=0)
#ax1 = df3.plot(x='start_time',y='TravelTime_s', style='.', color='r')
#ax1 = df3.plot(kind='scatter', x='start_time', y='TravelTime_s', style=['o', 'rx'], s=12)

#pd.scatter_matrix(df3, alpha=0.2, diagonal='kde')
df4.plot(x='start_time',y='mean', ax=ax1, lw=0.25)
#plt.yscale('log')

#plt.savefig('myplot.svg', facecolor='w')
plt.savefig('myplot.png', figsize=(60, 30), dpi=400, facecolor='w',s=1)
#plt.show()

In [6]:
df4.head(5)

,TripStart_dt,TripEnd_dt,Speed_kmh,TravelTime_s,start_time,day_num,mean
112373,2016-12-23 00:00:07,2016-12-23 00:05:58,12.666667,351,00:00:07,4,NaN
107515,2016-12-16 00:00:19,2016-12-16 00:18:33,4.063985,1094,00:00:19,4,NaN
138677,2017-02-03 00:00:22,2017-02-03 00:22:27,3.355472,1325,00:00:22,4,NaN
233540,2016-06-24 00:00:30,2016-06-24 00:27:08,2.782228,1598,00:00:30,4,NaN
233514,2016-06-24 00:00:30,2016-06-24 00:03:01,29.443709,151,00:00:30,4,NaN
